In [ ]:
!pip install googlemaps folium


  Preparing metadata (setup.py) ... done
  Created wheel for googlemaps: filename=googlemaps-4.10.0-py3-none-any.whl size=40716 sha256=c04e0c49d7f03c23b6c9cd7888dcf664c9e910a98dad46cfe2788d392678bdbe
  Stored in directory: /root/.cache/pip/wheels/17/f8/79/999d5d37118fd35d7219ef57933eb9d09886c4c4503a800f84
Successfully built googlemaps


In [ ]:
import googlemaps
import folium
from googlemaps.convert import decode_polyline
import time

In [ ]:
# Initialize the Google Maps client with your API key
gmaps = googlemaps.Client(replace_your_api_key)

In [ ]:
# Manually enter origin, destination, and buffer range
origin = input("Enter the starting location: ")
destination = input("Enter the destination location: ")
buffer_distance_km = float(input("Enter the buffer range in kilometers: "))

# Convert buffer distance from kilometers to meters for the Google API
buffer_distance_meters = buffer_distance_km * 1000  # Convert km to meters


Enter the starting location: Melbourne
Enter the destination location: Geelong
Enter the buffer range in kilometers: 10


In [ ]:
# Request directions with real-time traffic data
directions_result = gmaps.directions(origin, destination, mode="driving", departure_time='now')

In [ ]:
# Extract the polyline from the directions result
route_polyline = directions_result[0]['overview_polyline']['points']

# Decode the polyline into latitude/longitude coordinates
route_coords = decode_polyline(route_polyline)


In [ ]:
ev_charging_stations = []

# For each coordinate along the route, find nearby EV charging stations
for coord in route_coords:
    nearby_stations = gmaps.places_nearby(location=(coord['lat'], coord['lng']),
                                          radius=buffer_distance_meters,
                                          keyword='EV charging')
    ev_charging_stations.extend(nearby_stations['results'])


In [ ]:
while 'next_page_token' in nearby_stations:
    time.sleep(2)  # Wait for 2 seconds to avoid hitting the rate limit
    nearby_stations = gmaps.places_nearby(location=(coord['lat'], coord['lng']),
                                          radius=buffer_distance_meters,
                                          keyword='EV charging',
                                          page_token=nearby_stations['next_page_token'])
    ev_charging_stations.extend(nearby_stations['results'])


In [ ]:
unique_ev_charging_stations = {station['place_id']: station for station in ev_charging_stations}.values()

In [ ]:
station_eta_distance_list = []

# Calculate ETA and distance from origin to each charging station based on real-time traffic
for station in unique_ev_charging_stations:
    station_coords = (station['geometry']['location']['lat'], station['geometry']['location']['lng'])

    # Request directions from the origin to the station considering real-time traffic
    route_to_station = gmaps.directions(origin, station_coords, mode="driving", departure_time='now')

    # Extract the ETA (duration in traffic) and distance
    eta = route_to_station[0]['legs'][0]['duration_in_traffic']['value']  # ETA in seconds
    distance = route_to_station[0]['legs'][0]['distance']['value']  # Distance in meters
    station_eta_distance_list.append((station, eta, distance))



In [ ]:
# Sort charging stations by ETA (real-time traffic included)
station_eta_distance_list.sort(key=lambda x: x[1])  # Sort by ETA

# Get the nearest station (with the shortest ETA)
nearest_station, nearest_eta, nearest_distance = station_eta_distance_list[0]


In [ ]:
# Import necessary display functionality
from IPython.display import display

# Create a map centered on the route
map_center = [(route_coords[0]['lat'] + route_coords[-1]['lat']) / 2,
              (route_coords[0]['lng'] + route_coords[-1]['lng']) / 2]
map = folium.Map(location=map_center, zoom_start=10)

# Add the route to the map
folium.PolyLine([(point['lat'], point['lng']) for point in route_coords], color="blue", weight=2.5).add_to(map)

# Add EV charging stations to the map with ETA and distance
for station, eta, distance in station_eta_distance_list:
    lat = station['geometry']['location']['lat']
    lng = station['geometry']['location']['lng']
    name = station['name']
    folium.Marker([lat, lng], popup=f"{name} - ETA: {eta // 60} min - Distance: {distance / 1000:.2f} km").add_to(map)

# Display the map in Google Colab
display(map)

# Convert ETA from seconds to minutes for the nearest station
nearest_eta_minutes = nearest_eta // 60

# Convert distance from meters to kilometers for the nearest station
nearest_distance_km = nearest_distance / 1000

# Print the nearest station's name, ETA, and distance
print(f"Nearest EV Charging Station: {nearest_station['name']}")
print(f"Estimated Time: {nearest_eta_minutes} minutes")
print(f"Distance: {nearest_distance_km:.2f} kilometers")


Nearest EV Charging Station: Tesla Destination Charger
Estimated Time: 6 minutes
Distance: 1.46 kilometers
